## 5.1 여러 노드 운영을 위한 추가 사항 (node_network_1.ipynb)

In [2]:
import hashlib 
import json
from time import time
import random
import requests
from flask import Flask, request, jsonify

## Node network를 위해 추가로 import 되는 함수
from urllib.parse import urlparse



# Blockchain 객체 생성 
class Blockchain(object):
    
    def __init__(self):
        self.chain = []                                   # chain에 여러 block들 들어옴
        self.current_transaction = []                     # 임시 transaction 넣어줌
        self.nodes = set()                                # Node 목록을 보관
        self.new_block(previous_hash=1, proof=100)        # genesis block 생성

    @staticmethod
    def hash(block):
        block_string = json.dumps(block, sort_keys=True).encode() 
        return hashlib.sha256(block_string).hexdigest()   # hash 라이브러리로 sha256 사용
    @property
    def last_block(self):
        return self.chain[-1]                             # 체인의 마지막 블록 가져오기!!

    @staticmethod
    def valid_proof(last_proof, proof):
        guess = str(last_proof + proof).encode()          # 전 proof와 구할 proof 문자열 연결
        guess_hash = hashlib.sha256(guess).hexdigest()    # 이 hash 값 저장
        return guess_hash[:4] == "0000"                  # 앞 4자리가 0000 이면 True (알맞은 nonce값을 찾음)

    def pow(self, last_proof):
        proof = random.randint(-1000000,1000000)
        while self.valid_proof(last_proof, proof) is False: # valid proof 함수 활용(아래 나옴), 맞을 때까지 반복적으로 검증
            proof = random.randint(-1000000,1000000)
        print("final+proof : ", proof)
        return proof

    def new_transaction(self, sender, recipient, amount):
        self.current_transaction.append(
            {
                'sender' : sender, # 송신자
                'recipient' : recipient, # 수신자
                'amount' : amount, # 금액
                'timestamp':time()
            }
        )
        return self.last_block['index'] + 1   

    def new_block(self, proof, previous_hash=None):
        block = {
            'index' : len(self.chain)+1,
            'timestamp' : time(), # timestamp from 1970
            'transactions' : self.current_transaction,
            'nonce' : proof,
            'previous_hash' : previous_hash or self.hash(self.chain[-1]),
        }
        self.current_transaction = []
        self.chain.append(block)     
        return block

    def valid_chain(self, chain):
        last_block = chain[0] 
        current_index = 1

        while current_index < len(chain): 
            block = chain[current_index]
            print('%s' % last_block)
            print('%s' % block)
            print("\n--------\n")
            if block['previous_hash'] != self.hash(last_block):
                return False
            last_block = block
            current_index += 1
        return True
    
    # 노드 등록
    # 객체 내의 노드 리스트에 다른 노드 추가하기
    def register_node(self, address):
        parsed_url = urlparse(address)
        self.nodes.add(parsed_url.netloc)

    # 노드의 블록 유효성 검증
    # 다른 블록과 데이터를 비교하며 최신 데이터로 업데이트
    def resolve_conflicts(self):
        neighbors = self.nodes      # 구동되는 노드들을 저장
        new_block = None

        max_length = len(self.chain)    # 내 블록의 길이 저장
        for node in neighbors:
            node_url = "http://" + str(node.replace("0.0.0.0", "localhost")) + '/chain'     # url을 받아서 request 통해 체인 정보 저장
            response = requests.get(node_url)
            if response.status_code == 200: # 웹페이지와 정상적으로 교류가 되면 그 정보 저장
                length = response.json()['length']
                chain = response.json()['chain']
                ## 다른 노드의 길이(length)가 내 노드의 길이(max_length)보다 길고 and 내 체인이 유효한 경우
                if length > max_length and self.valid_chain(chain): # 긴 체인을 비교 >> 제일 긴 블록이 인정된다
                    max_length = length
                    ## 기존 노드의 정보보다 받은 정보가 최신이다. 전송받은 블록 정보를 new_block에 넣는다
                    new_block = chain
                ## 다른 노드의 길이(length)가 내 노드의 길이(max_length)보다 짧거나 내 체인이 유효하지 않은 경우
                else:
                    1==1    # 별도 작업 불필요
                
            if new_block != None:
                self.chain = new_block  # 기존 블록 정보가 잘못된 것을 인정하고 검증된 블록 정보로 바꾼다. 



### Blockchain 객체 기반으로 노드 생성 ###
# 노드 기본값 설정
blockchain = Blockchain()
my_ip = 'localhost'
my_port = '5000'
node_identifier = 'node_' + my_port
mine_owner = 'master'
mine_profit = 0.1

app = Flask(__name__)


@app.route('/chain', methods=['GET'])
def full_chain():
    print("chain info requested!!")
    response = {
        'chain' : blockchain.chain, 
        'length' : len(blockchain.chain), 
    }
    return jsonify(response), 200


## 노드 연결을 위해 추가되는 함수 : 다른 Node 등록
@app.route('/nodes/register', methods=['POST'])
def register_nodes():
    values = request.get_json() #json 형태로 보내면 노드가 저장됨
    print("register nodes!!!", values)
    registering_node = values.get('nodes')
    if registering_node == None:   # 요청된 node 값이 없다면!
        return "Error: Please supply a valid list of nodes", 400
    
    ## 요청받은 노드가 이미 등록된 노드와 중복인지 검사
    ## 중복인 경우 
    if registering_node.split('//')[1] in blockchain.nodes:
        print("Node already registered")    # 이미 등록된 노드입니다.
        response = {
            'message' : 'Already Registered Node',
            'total_nodes' : list(blockchain.nodes),
        }

    ## 중복이 아니라면
    else:
        # 내 노드 리스트에 추가
        blockchain.register_node(registering_node)

        ## 이후 해당 노드에 내 정보 등록하기
        headers = {'Content-Type': 'application/json; charset=utf-8'}
        data = {
            "nodes" : 'http://' + my_ip + ":" + my_port
        }
        print("MY NODE INFO ", 'http://' + my_ip + ":" + my_port)
        requests.post(registering_node + "nodes/register", headers=headers, data = json.dumps(data))

    ## 이후 주변 노드들에도 새로운 노드가 등장함을 전파
    for add_node in blockchain.nodes:
        if add_node != registering_node.split("//")[1]:
            print('add_node : ', add_node)
            ## 노드 등록하기 
            headers = {'Content-Type' : 'application/json; charset=utf-8'}
            data = {
                "nodes" : registering_node
            }
            requests.post('http://' + add_node + "/nodes/register", headers=headers, data = json.dumps(data))

    response = {
        'message' : 'New nodes have been added',
        'total_nodes' : list(blockchain.nodes),
    }

    return jsonify(response), 201
